# LangChain サンプル 3: Memory

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** このノートブックは、SageMaker Studioの **Data Science 3.0** カーネルで動作します

### LangChain の Memory を使わない生成 AI のチャットボット

In [ ]:
from langchain_aws import ChatBedrock                   
from langchain_core.messages.human import HumanMessage 

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

print("私は LangChain の Memory モジュールを使わないチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        result = chat.invoke( # Chat modelsを使ってモデルを呼び出す
            [
              HumanMessage(content = prompt) 
            ]
        )

        # レスポンスを表示
        print("response> " + result.content)
        
print("チャットボットを終了しました。\n")

###  LangChain の Memory を使った生成 AI のチャットボット

In [ ]:
from langchain_aws import ChatBedrock                   
from langchain.memory.buffer import ConversationBufferMemory  
from langchain_core.messages.human import HumanMessage    

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

memory = ConversationBufferMemory( # メモリを初期化
    return_messages = True
)

print("私は LangChain の ConversationBufferMemory を使っているチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        memory_message_result = memory.load_memory_variables({}) # メモリの内容を取得
        messages = memory_message_result['history'] # メモリの内容からメッセージのみを取得
        messages.append(HumanMessage(content=prompt)) # ユーザーからのメッセージを追加

        result = chat.invoke( # Chat modelsを使ってモデルを呼び出す
            messages
        )
        
        memory.save_context(  # メモリにメッセージを追加
            {
                "input": prompt,  # ユーザーからのメッセージをinputとして保存
            },
            {
                "output": result.content,  # AIからのメッセージをoutputとして保存
            }
        )
        # レスポンスを表示
        print("response> " + result.content)
        
print("チャットボットを終了しました。\n")

###  LangChain の ConversationChain を使った生成 AI のチャットボット

In [ ]:
from langchain.chains.conversation.base import ConversationChain  
from langchain_aws import ChatBedrock                   
from langchain.memory.buffer import ConversationBufferMemory  
from langchain_core.messages.human import HumanMessage    

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

memory = ConversationBufferMemory( 
    return_messages = True
)

chain = ConversationChain( # ConversationChainを初期化
    memory = memory,
    llm = chat,
)

print("私は LangChain の ConversationChain を使っているチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        result = chain.invoke(prompt)
        # レスポンスを表示
        print("response> " + result["response"])
print("チャットボットを終了しました。\n")

###  LangChain の ConversationBufferWindowMemory を使った生成 AI のチャットボット

In [ ]:
from langchain.memory.buffer_window import ConversationBufferWindowMemory 
from langchain.chains.conversation.base import ConversationChain  
from langchain_aws import ChatBedrock                   
from langchain_core.messages.human import HumanMessage    

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

memory = ConversationBufferWindowMemory(
    return_messages = True,
    k = 3 # 3往復分のメッセージを記憶
)

chain = ConversationChain(
    memory=memory,
    llm=chat
)

print("私は LangChain の ConversationBufferWindowMemory を使っているチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        messages = chain.memory.load_memory_variables({})["history"] # 保存されているメッセージを取得
        print(f"--- 保存されているメッセージの数: {len(messages)} ---") # 保存されているメッセージ数を表示

        for saved_message in messages: # 保存されているメッセージを1つずつ取り出す
            print(saved_message.content) # 保存されているメッセージを表示する
        print("---" * 10) 

        result = chain.invoke(prompt)
        # レスポンスを表示
        print(result["response"])
        
print("チャットボットを終了しました。\n")

###  LangChain の ConversationSummaryMemory を使った生成 AI のチャットボット

In [ ]:
from langchain.chains.conversation.base import ConversationChain  
from langchain_aws import ChatBedrock                           
from langchain.memory.summary import ConversationSummaryMemory  
from langchain_core.messages.human import HumanMessage    
from langchain_core.messages.system import SystemMessage  


chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

memory = ConversationSummaryMemory(  # ConversationSummaryMemoryを使用するように変更
    llm=chat,  # Chat modelsを指定
    return_messages=True,
)

chain = ConversationChain(
    memory=memory,
    llm=chat,
)

print("私は LangChain の ConversationBufferWindowMemory を使っているチャットボットです。メッセージを入力してください。\n")

flag = True

while flag:
    prompt = input("prompt>")
    
    if prompt == "quit":
        flag = False
    else:
        messages = chain.memory.load_memory_variables({})["history"] # 保存されているメッセージを取得

        print(f"--- 保存されているメッセージの数: {len(messages)} ---") # 保存されているメッセージの数を表示
        for saved_message in messages: # 保存されているメッセージを1つずつ取り出す
            print(saved_message.content) # 保存されているメッセージを表示する
        print("---" * 10) 

        result = chain.invoke(prompt)
        # レスポンスを表示
        print(result["response"])
        
print("チャットボットを終了しました。\n")